In [87]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier


In [56]:
dir_test = "../Data/test.csv"
dir_train = "../Data/train.csv"

df_Test = pd.read_csv(dir_test)
df_Train = pd.read_csv(dir_train)

In [57]:
print(df_Train.shape)
print(df_Test.shape)

(891, 12)
(418, 11)


VER SI HAY DATOS FALTANTES

In [58]:
print(pd.isnull(df_Train).sum())
print(pd.isnull(df_Test).sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [59]:
print(df_Train.describe())
print(df_Test.describe())

       PassengerId    Survived      Pclass         Age       SibSp   
count   891.000000  891.000000  891.000000  714.000000  891.000000  \
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   20.125000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   38.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200  
       PassengerId      Pclass         Age       SibSp       Parch        Fare
co

PREPROCESAR LOS DATOS

In [63]:
#Cambiar de sexo a numeros 
df_Train['Sex'].replace(['female','male'],[0,1],inplace=True)
df_Test['Sex'].replace(['female','male'],[0,1],inplace=True)

In [64]:
#Cambiar el embarque a numero
df_Train['Embarked'].replace(['Q','S','C'],[0,1,2],inplace=True)
df_Test['Embarked'].replace(['Q','S','C'],[0,1,2],inplace=True)

In [67]:
#Reemplaza los datos faltantes de las edades por la media
print(df_Train['Age'].mean())
print(df_Test['Age'].mean())
prom=30 
df_Train['Age']=df_Train['Age'].replace(np.nan, prom)
df_Test['Age']=df_Test['Age'].replace(np.nan, prom)
#print(pd.isnull(df_Test).sum())

29.758888888888887
30.216507177033492
PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [68]:
#Crear grupos de acuerdo a las edades, crear bandas y reemplazarlas
bins=[0,10,20,30,40,50,60,70,80,90,100]
names=['1','2','3','4','5','6','7','8','9','10']
df_Train['Age']=pd.cut(df_Train['Age'], bins, labels = names)
df_Train['Age']=pd.cut(df_Test['Age'], bins, labels = names)

In [69]:
#Se eliminar la columna de "Cabin" ya que tiene muchos datos perdidos 
#Y no se pueden reemplazar ya que son inexactos
df_Train.drop(['Cabin'],axis = 1,inplace=True)
df_Test.drop(['Cabin'],axis = 1,inplace=True)

In [70]:
print(pd.isnull(df_Test).sum())

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           1
Embarked       0
dtype: int64


ELIMINAR COLUMNAS QUE NOS SON NECESARIAS PARA EL ANALISIS
ID de pasajero, Nombre, Ticket

In [78]:
df_Train=df_Train.drop(['PassengerId','Name','Ticket'], axis=1)
df_Test=df_Test.drop(['Name','Ticket'], axis=1)


.


In [79]:
#Eliminar filas con datos perdidos 
df_Train.dropna(axis=0, how='any', inplace=True)
df_Test.dropna(axis=0, how='any', inplace=True)

VERIFICACION DE DATA

In [82]:
print(pd.isnull(df_Train).sum())
print(pd.isnull(df_Test).sum())

print(df_Train.shape)
print(df_Test.shape)

print(df_Train.head())
print(df_Test.head())

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64
PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64
(417, 8)
(417, 10)
   Survived  Pclass  Sex Age  SibSp  Parch     Fare  Embarked
0         0       3    1   4      1      0   7.2500       1.0
1         1       1    0   5      1      0  71.2833       2.0
2         1       3    0   7      0      0   7.9250       1.0
3         1       1    0   3      1      0  53.1000       1.0
4         0       3    1   3      0      0   8.0500       1.0
   PassengerId  Pclass                                          Name  Sex   
0          892       3                              Kelly, Mr. James    1  \
1          893       3              Wilkes, Mrs. James (Ellen Needs)    0   
2          894       2                     Myles, Mr. Thomas Francis  

ENTRENARLO 

In [104]:
#Separa la columna con la informacion de los sobrevivientes
x = np.array(df_Train.drop('Survived', axis=1))
y = np.array(df_Train['Survived'])

In [105]:
#Separa los datos train en entrenamiento y prueba para los algoritmos
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [106]:
#Regresion logistica 
logreg=LogisticRegression()
logreg.fit(x_train, y_train)
y_pred=logreg.predict(x_test)
print('Precision Regresión Logística: ')
print(logreg.score(x_train, y_train))

Precision Regresión Logística: 
0.8108108108108109


In [107]:
#Super Vector Machines
svc=SVC()
svc.fit(x_train, y_train)
y_pred=svc.predict(x_test)
print('Precision Soporte de Vectores: ')
print(svc.score(x_train, y_train))

Precision Soporte de Vectores: 
0.6426426426426426


In [108]:
#K Neighbors
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(x_train, y_train)
y_pred=knn.predict(x_test)
print('Precision Soporte de Vectores: ')
print(knn.score(x_train, y_train))

Precision Soporte de Vectores: 
0.7927927927927928


PREDICCIÓN

In [109]:
ids=df_Test['PassengerId']

In [115]:
#Regresion logistica
prediccion_logreg=logreg.predict(df_Test.drop('PassengerId', axis=1))
out_logreg=pd.DataFrame({'PassengerId' :ids, 'Survived' :prediccion_logreg})
print('Prediccion Regresion Logistica: ')
print(out_logreg.head())

C:\Users\User\miniconda3\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


ValueError: could not convert string to float: 'Kelly, Mr. James'

In [114]:
df_Test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 417 entries, 0 to 417
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  417 non-null    int64  
 1   Pclass       417 non-null    int64  
 2   Name         417 non-null    object 
 3   Sex          417 non-null    int64  
 4   Age          417 non-null    float64
 5   SibSp        417 non-null    int64  
 6   Parch        417 non-null    int64  
 7   Ticket       417 non-null    object 
 8   Fare         417 non-null    float64
 9   Embarked     417 non-null    int64  
dtypes: float64(2), int64(6), object(2)
memory usage: 35.8+ KB
